Das brauche ich um's nicht zu vergessen.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


1. Originaldaten importieren

In [59]:
df = pd.read_csv('/Users/whypk/01Projekte/air_quality/data/waqi-covid19-airqualitydata-2025.csv', comment='#')
df.sample(50)
df.shape

(2321072, 9)

2. Spalten löschen
Wir konzentrieren uns bei der Analyse auf den Median der jeweiligen Messungen, daher kann auf die Spalten min und max verzichtet werden.
Die Berechnung der Varianz können wir nicht nachvollziehen und verzichten auch darauf.

In [46]:
df_small_24 = df.drop(columns=['variance', 'min', 'max'])
df_small_24.sample(20)

,Date,Country,City,Specie,count,median
430662,2024-08-28,FR,Toulouse,wind-speed,224,3.0
1629376,2024-12-02,CN,Wuhan,co,209,10.9
371608,2024-02-19,DE,Düsseldorf,o3,69,19.9
1552942,2024-01-08,CN,Lhasa,pressure,30,654.5
2028903,2024-10-07,NZ,Hamilton,precipitation,12,0.8
1056672,2024-11-16,RO,Sibiu,no2,17,1.6
360726,2025-01-15,DE,Dresden,so2,2,0.3
1228789,2024-09-16,KR,Incheon,pm25,480,17.0
834698,2024-11-19,IN,Ghāziābād,wind-speed,48,0.3
919198,2024-11-21,HU,Pécs,o3,48,17.3


In [61]:
#df_small_24.nunique('City')
df_small_24['City'].nunique()

579

3. Eine Stadt/ Land und zwar die mit den meisten Messwerten.

In [47]:
# Which cities per country have the highest number of measurements?
city_counts = df_small_24.groupby(["Country", "City"]).size().reset_index(name="count")
city_counts.sample(10)

,Country,City,count
313,JP,Kochi,4575
386,MY,Kota Bharu,2610
401,NL,Nijmegen,3671
5,AT,Linz,4805
357,KR,Yeosu,4891
74,CN,Fushun,4325
415,NZ,Hamilton,3623
7,AT,Vienna,4849
565,VN,Haiphong,2409
301,IT,Turin,4037


In [48]:
# Für jedes Land die Stadt mit den meisten Einträgen finden
max_city_per_country = city_counts.loc[city_counts.groupby("Country")["count"].idxmax()]
max_city_per_country.sample(10)


,Country,City,count
464,SA,Dammam,385
481,TM,Ashgabat,1958
440,PT,Lisbon,3994
288,IR,Tehran,4163
119,CW,Willemstad,818
363,LT,Kaunas,3857
303,JO,Irbid,4507
564,UZ,Tashkent,2964
24,BE,Brussels,3992
347,KR,Jeju City,5138


In [49]:
# Ergebnisse anzeigen
print(max_city_per_country)
#als csv ablegen
max_city_per_country.to_csv('/Users/whypk/01Projekte/air_quality/data/max_city_per_country.csv', index=False)

    Country          City  count
0        AE     Abu Dhabi   4571
2        AR  Buenos Aires   4103
7        AT        Vienna   4849
9        AU      Brisbane   5599
19       BA      Sarajevo   4665
..      ...           ...    ...
519      US        Dallas   5143
564      UZ      Tashkent   2964
566      VN         Hanoi   4502
570      XK      Pristina   3555
578      ZA      Pretoria   4872

[90 rows x 3 columns]


4. Die ausgewählen Städte im df selektieren

In [ ]:
# Aus max_city_per_country die Städte als Liste speichern
city_list_24=df_small_24['City'].tolist()

#aus df_small_24 nur die gewünschten Städte filtern
df_small_24_filtered = df_small_24[df_small_24['City'].isin(city_list_24)]
df_small_24_filtered

,Date,Country,City,Specie,count,median
0,2024-12-16,PE,Lima,o3,120,1.7
1,2025-01-04,PE,Lima,o3,192,2.6
2,2024-02-06,PE,Lima,o3,167,4.9
3,2024-04-19,PE,Lima,o3,144,5.4
4,2024-08-15,PE,Lima,o3,97,2.8
...,...,...,...,...,...,...
2321067,2024-03-19,IQ,Baghdad,wind-gust,2,13.8
2321068,2024-04-09,IQ,Baghdad,wind-gust,2,15.9
2321069,2024-06-25,IQ,Baghdad,wind-gust,2,9.7
2321070,2024-12-27,IQ,Baghdad,wind-gust,2,16.4


In [52]:
mask_top_five_cities = (df['City'] == "Abu Dhabi") | (df['City'] == "Buenos Aires") | (df['City'] == "Vienna") | (df['City'] == "Brisbane") | (df['City'] == "Sarajevo")
df_top_five_cities = df[mask_top_five_cities]
df_top_five_cities.sample(50)

,Date,Country,City,Specie,count,min,max,median,variance
1366401,2024-06-16,BA,Sarajevo,wind-speed,144,0.7,5.1,1.5,14.80
1951129,2024-05-29,AE,Abu Dhabi,o3,67,1.0,57.8,31.6,1520.91
1950975,2024-03-05,AE,Abu Dhabi,humidity,192,60.0,94.0,73.0,662.85
1366536,2024-02-22,BA,Sarajevo,wind-speed,144,0.2,6.6,1.0,33.66
997742,2024-12-20,AT,Vienna,pressure,360,978.4,1027.2,1016.5,527.88
1949306,2024-02-03,AE,Abu Dhabi,dew,168,8.0,12.0,10.0,6.19
320943,2024-02-22,AU,Brisbane,temperature,206,21.7,34.2,27.1,94.46
318992,2024-01-13,AU,Brisbane,wd,216,57.0,240.0,146.0,13644.40
1363430,2024-09-18,BA,Sarajevo,humidity,144,66.7,100.0,96.5,683.07
318561,2024-05-01,AU,Brisbane,pressure,210,1014.9,1022.6,1020.8,31.80


In [53]:
mask_vienna = (df['City'] == "Vienna") & (df['Date'] == "2024-01-01")& (df['Country'] == "AT")
df_vienna = df[mask_vienna]
print(df_vienna)

               Date Country    City         Specie  count    min     max  \
996426   2024-01-01      AT  Vienna           pm10    356    1.0    49.0   
996763   2024-01-01      AT  Vienna       humidity    408   57.3    95.8   
997373   2024-01-01      AT  Vienna            so2     59    0.6    51.7   
997467   2024-01-01      AT  Vienna       pressure    408  983.9  1021.2   
997863   2024-01-01      AT  Vienna  precipitation     36    0.1    99.6   
998235   2024-01-01      AT  Vienna     wind-speed    402    0.2    22.2   
998702   2024-01-01      AT  Vienna             co     36    0.1     0.1   
999348   2024-01-01      AT  Vienna    temperature    408    2.8     9.4   
999713   2024-01-01      AT  Vienna            no2    351    1.0    19.9   
1000017  2024-01-01      AT  Vienna      wind-gust    336    0.8    43.5   
1000420  2024-01-01      AT  Vienna            dew    249   -0.5     3.4   
1000735  2024-01-01      AT  Vienna           pm25    296    5.0   107.0   

         me

In [54]:
mask_london = (df['City'] == "London") & (df['Date'] == "2025-01-08")& (df['Country'] == "GB")
df_london = df[mask_london]
print(df_london)

               Date Country    City       Specie  count    min     max  \
2240923  2025-01-08      GB  London   wind-speed    473    0.1     5.7   
2241376  2025-01-08      GB  London    wind-gust    349    0.2    10.8   
2241510  2025-01-08      GB  London           co    192    1.5     6.8   
2241860  2025-01-08      GB  London  temperature    576   -2.0     3.0   
2242542  2025-01-08      GB  London         pm10    483    2.0    52.0   
2242637  2025-01-08      GB  London     humidity    576   80.0    99.0   
2243236  2025-01-08      GB  London          no2    462    4.3    38.0   
2243853  2025-01-08      GB  London          dew     72   -1.0     1.0   
2244234  2025-01-08      GB  London           o3    226    0.6    20.8   
2244584  2025-01-08      GB  London          so2    167    0.3     2.7   
2244883  2025-01-08      GB  London         pm25    479    5.0    97.0   
2245376  2025-01-08      GB  London     pressure    576  997.2  1030.7   

         median  variance  
2240923  

In [55]:
#df_london = df_london.drop(columns=['variance', 'min', 'max'])
df_london

,Date,Country,City,Specie,count,min,max,median,variance
2240923,2025-01-08,GB,London,wind-speed,473,0.1,5.7,2.2,19.42
2241376,2025-01-08,GB,London,wind-gust,349,0.2,10.8,5.3,79.36
2241510,2025-01-08,GB,London,co,192,1.5,6.8,4.6,24.95
2241860,2025-01-08,GB,London,temperature,576,-2.0,3.0,1.1,12.07
2242542,2025-01-08,GB,London,pm10,483,2.0,52.0,14.0,391.86
2242637,2025-01-08,GB,London,humidity,576,80.0,99.0,90.0,140.53
2243236,2025-01-08,GB,London,no2,462,4.3,38.0,21.5,492.28
2243853,2025-01-08,GB,London,dew,72,-1.0,1.0,0.0,5.42
2244234,2025-01-08,GB,London,o3,226,0.6,20.8,6.0,266.90
2244584,2025-01-08,GB,London,so2,167,0.3,2.7,1.0,2.54


In [56]:
df_pivot_london = df_london.pivot(index=["Date", "Country", "City"], columns="Specie", values=["median", 'count']).reset_index()
df_pivot_london.columns = ['_'.join(col).strip('_') for col in df_pivot_london.columns]
df_pivot_london

,Date,Country,City,median_co,median_dew,median_humidity,median_no2,median_o3,median_pm10,median_pm25,...,count_humidity,count_no2,count_o3,count_pm10,count_pm25,count_pressure,count_so2,count_temperature,count_wind-gust,count_wind-speed
0,2025-01-08,GB,London,4.6,0.0,90.0,21.5,6.0,14.0,39.0,...,576.0,462.0,226.0,483.0,479.0,576.0,167.0,576.0,349.0,473.0


In [57]:
df_top_five_cities_pivot = df_top_five_cities.pivot(index=["Date", "Country", "City"], columns="Specie", values=["median", 'count']).reset_index()
df_top_five_cities_pivot.columns = ['_'.join(col).strip('_') for col in df_top_five_cities_pivot.columns]
df_top_five_cities_pivot


,Date,Country,City,median_co,median_dew,median_humidity,median_no2,median_o3,median_pm10,median_pm25,...,count_o3,count_pm10,count_pm25,count_precipitation,count_pressure,count_so2,count_temperature,count_wd,count_wind-gust,count_wind-speed
0,2024-01-01,AE,Abu Dhabi,0.1,19.0,73.0,8.1,21.2,50.0,53.0,...,119.0,120.0,24.0,NaN,192.0,120.0,192.0,NaN,NaN,184.0
1,2024-01-01,AR,Buenos Aires,1.9,15.8,63.0,4.6,12.2,14.0,34.0,...,24.0,75.0,24.0,2.0,92.0,8.0,92.0,21.0,48.0,92.0
2,2024-01-01,AT,Vienna,0.1,1.9,72.7,4.2,NaN,6.0,17.0,...,NaN,356.0,296.0,36.0,408.0,59.0,408.0,NaN,336.0,402.0
3,2024-01-01,AU,Brisbane,1.5,20.4,87.0,1.9,10.6,9.0,15.0,...,72.0,215.0,212.0,102.0,216.0,5.0,208.0,216.0,26.0,216.0
4,2024-01-01,BA,Sarajevo,1.6,6.0,96.5,15.6,25.2,21.0,76.0,...,144.0,121.0,98.0,6.0,144.0,115.0,144.0,NaN,6.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2025-02-17,AR,Buenos Aires,NaN,19.4,87.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,26.0,NaN,26.0,NaN,NaN,13.0
2053,2025-02-17,AT,Vienna,0.1,-9.0,70.2,9.6,NaN,26.0,76.0,...,NaN,235.0,216.0,NaN,265.0,43.0,265.0,NaN,152.0,157.0
2054,2025-02-17,AU,Brisbane,0.7,16.4,58.5,2.3,8.6,23.0,27.0,...,69.0,216.0,216.0,NaN,203.0,19.0,214.0,192.0,24.0,214.0
2055,2025-02-17,BA,Sarajevo,3.7,-4.0,86.0,12.4,15.6,31.0,91.0,...,86.0,102.0,101.0,NaN,107.0,102.0,107.0,NaN,NaN,107.0
